<a href="https://colab.research.google.com/github/SanthoshToorpu/Audio-Classififcation/blob/main/Bigram_unoptimized.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
# read it in to inspect it
with open('/content/drive/MyDrive/pride_and_prejudice.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [37]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  726907


In [38]:
# let's look at the first 1000 characters
print(text[:1000])

PREFACE.


_Walt Whitman has somewhere a fine and just distinction between “loving
by allowance” and “loving with personal love.” This distinction applies
to books as well as to men and women; and in the case of the not very
numerous authors who are the objects of the personal affection, it
brings a curious consequence with it. There is much more difference as
to their best work than in the case of those others who are loved “by
allowance” by convention, and because it is felt to be the right and
proper thing to love them. And in the sect--fairly large and yet
unusually choice--of Austenians or Janites, there would probably be
found partisans of the claim to primacy of almost every one of the
novels. To some the delightful freshness and humour of_ Northanger
Abbey, _its completeness, finish, and_ entrain, _obscure the undoubted
critical facts that its scale is small, and its scheme, after all, that
of burlesque or parody, a kind in which the first rank is reached with
difficulty._ Pers

In [39]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !&()*,-./0123456789:;?ABCDEFGHIJKLMNOPRSTUVWXYZ[]^_abcdefghijklmnopqrstuvwxyz{}·àâéêœ‘’“”
91


In [40]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[60, 61, 61, 1, 72, 60, 57, 70, 57]
hii there


In [41]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([726907]) torch.int64
tensor([39, 40, 28, 29, 24, 26, 28,  9,  0,  0,  0, 52, 45, 53, 64, 72,  1, 45,
        60, 61, 72, 65, 53, 66,  1, 60, 53, 71,  1, 71, 67, 65, 57, 75, 60, 57,
        70, 57,  1, 53,  1, 58, 61, 66, 57,  1, 53, 66, 56,  1, 62, 73, 71, 72,
         1, 56, 61, 71, 72, 61, 66, 55, 72, 61, 67, 66,  1, 54, 57, 72, 75, 57,
        57, 66,  1, 89, 64, 67, 74, 61, 66, 59,  0, 54, 77,  1, 53, 64, 64, 67,
        75, 53, 66, 55, 57, 90,  1, 53, 66, 56,  1, 89, 64, 67, 74, 61, 66, 59,
         1, 75, 61, 72, 60,  1, 68, 57, 70, 71, 67, 66, 53, 64,  1, 64, 67, 74,
        57,  9, 90,  1, 42, 60, 61, 71,  1, 56, 61, 71, 72, 61, 66, 55, 72, 61,
        67, 66,  1, 53, 68, 68, 64, 61, 57, 71,  0, 72, 67,  1, 54, 67, 67, 63,
        71,  1, 53, 71,  1, 75, 57, 64, 64,  1, 53, 71,  1, 72, 67,  1, 65, 57,
        66,  1, 53, 66, 56,  1, 75, 67, 65, 57, 66, 22,  1, 53, 66, 56,  1, 61,
        66,  1, 72, 60, 57,  1, 55, 53, 71, 57,  1, 67, 58,  1, 72, 60, 57,  1,
       

In [42]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [43]:
block_size = 8
train_data[:block_size+1]

tensor([39, 40, 28, 29, 24, 26, 28,  9,  0])

In [44]:
# **1. Slice the training data:**
x = train_data[:block_size]  # Create a slice of `block_size` elements from the beginning of `train_data`
y = train_data[1:block_size+1]  # Create a slice starting from the second element to the `block_size+1` element

# **2. Iterate through the blocks:**
for t in range(block_size):
    # **3. Construct context and target:**
    context = x[:t+1]  # Subset of `x` up to (and including) the current index `t`
    target = y[t]  # Element at index `t` in `y`

    # **4. Print context and target:**
    print(f"when input is {context} the target: {target}")


when input is tensor([39]) the target: 40
when input is tensor([39, 40]) the target: 28
when input is tensor([39, 40, 28]) the target: 29
when input is tensor([39, 40, 28, 29]) the target: 24
when input is tensor([39, 40, 28, 29, 24]) the target: 26
when input is tensor([39, 40, 28, 29, 24, 26]) the target: 28
when input is tensor([39, 40, 28, 29, 24, 26, 28]) the target: 9
when input is tensor([39, 40, 28, 29, 24, 26, 28,  9]) the target: 0


In [45]:
# **1. Set seed for reproducibility:**
torch.manual_seed(1337)  # Ensure consistent random number generation

# **2. Define hyperparameters:**
batch_size = 4  # Number of sequences processed in parallel
block_size = 8  # Maximum context length for predictions

# **3. Function to generate batches of data:**
def get_batch(split):
    # Select appropriate dataset based on split
    data = train_data if split == 'train' else val_data

    # Randomly select batch_size starting indices for sequences
    ix = torch.randint(len(data) - block_size, (batch_size,))

    # Create input and target sequences using stacking
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])

    # Return input and target batches
    return x, y

# **4. Generate a batch of training data:**
xb, yb = get_batch('train')

# **5. Print input and target shapes and values:**
print('inputs:')
print(xb.shape)  # Print dimensions of input batch
print(xb)        # Print actual input batch values
print('targets:')
print(yb.shape)  # Print dimensions of target batch
print(yb)        # Print actual target batch values

# **6. Iterate through the batch and time dimensions:**
print('----')
for b in range(batch_size):  # Loop through each sequence in the batch
    for t in range(block_size):  # Loop through each time step in the sequence
        # Extract context and target for current position
        context = xb[b, :t+1]  # Slice input up to current time step
        target = yb[b, t]      # Corresponding target element

        # Print context and target
        print(f"when input is {context.tolist()} the target: {target}")


inputs:
torch.Size([4, 8])
tensor([[ 8,  0,  0, 89, 32,  1, 60, 53],
        [ 9,  1, 42, 60, 53, 72,  1, 75],
        [70, 53, 54, 64, 57,  9, 90,  0],
        [61, 72,  1, 61, 71,  1, 62, 73]])
targets:
torch.Size([4, 8])
tensor([[ 0,  0, 89, 32,  1, 60, 53, 74],
        [ 1, 42, 60, 53, 72,  1, 75, 61],
        [53, 54, 64, 57,  9, 90,  0,  0],
        [72,  1, 61, 71,  1, 62, 73, 71]])
----
when input is [8] the target: 0
when input is [8, 0] the target: 0
when input is [8, 0, 0] the target: 89
when input is [8, 0, 0, 89] the target: 32
when input is [8, 0, 0, 89, 32] the target: 1
when input is [8, 0, 0, 89, 32, 1] the target: 60
when input is [8, 0, 0, 89, 32, 1, 60] the target: 53
when input is [8, 0, 0, 89, 32, 1, 60, 53] the target: 74
when input is [9] the target: 1
when input is [9, 1] the target: 42
when input is [9, 1, 42] the target: 60
when input is [9, 1, 42, 60] the target: 53
when input is [9, 1, 42, 60, 53] the target: 72
when input is [9, 1, 42, 60, 53, 72] the targ

In [46]:
print(xb) # our input to the transformer

tensor([[ 8,  0,  0, 89, 32,  1, 60, 53],
        [ 9,  1, 42, 60, 53, 72,  1, 75],
        [70, 53, 54, 64, 57,  9, 90,  0],
        [61, 72,  1, 61, 71,  1, 62, 73]])


In [47]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 91])
tensor(4.5440, grad_fn=<NllLossBackward0>)

RàYb?LtyjP3Ae3JVlElp0UYy”6hKD}:hs7’iMêRkwkdbli47-”:NiyScàqD9‘k“)GS?s)Llj
HmâF1blâOyjv:à^w[lwk“)U7dfa


In [52]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-4)

In [53]:
batch_size = 32
for steps in range(10000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


2.3633055686950684


In [54]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


he akik, Winoby
l mis Teint ancaty estovoowsore otos wowie he s gnd tim
w]Jabe
pasher t atotedivint.”
odeasster wan nanlinicoust ales he arger bacoutwhat,
“iss jercth. he thexpt Swan owow ond r s the stiry hearf  se ecervenenerber”

tovemine “a atovent I aban f Bist,” o
or silinisssg-ive ke
wiba orase?”
at yPecy t; terer leroty, daitepe s
t  sind aby dose oo  AGe--.

ns in, song w  f cthe  bemaneishithier wnndoutor. gus asso o
Mrlen

hiveJan istavend I hig  is cl rdeore d
hep thimot n I  s  towe
